In [5]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

x = torch.rand(10000, 10000, device=device)  # allocated on GPU
print(x.device)


CUDA available: True
GPU name: NVIDIA GeForce RTX 4060 Laptop GPU
cuda:0


In [6]:
pip install langchain langchain-community


  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.1.0-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 17.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 25.4 MB/s eta 0:00:00
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)
Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
Using cached marshmallow-3.26.1-py3-none-any.whl (50 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
pip install llama-cpp-python

     ---------------------------------------- 0.0/50.7 MB ? eta -:--:--
     -- ------------------------------------- 3.4/50.7 MB 20.2 MB/s eta 0:00:03
     ------ --------------------------------- 8.7/50.7 MB 22.1 MB/s eta 0:00:02
     ---------- ---------------------------- 13.4/50.7 MB 22.4 MB/s eta 0:00:02
     --------------- ----------------------- 20.2/50.7 MB 25.9 MB/s eta 0:00:02
     --------------------- ----------------- 28.0/50.7 MB 27.7 MB/s eta 0:00:01
     ------------------------- ------------- 32.8/50.7 MB 27.3 MB/s eta 0:00:01
     --------------------------- ----------- 35.4/50.7 MB 25.2 MB/s eta 0:00:01
     ----------------------------- --------- 38.0/50.7 MB 23.9 MB/s eta 0:00:01
     ------------------------------- ------- 40.9/50.7 MB 22.5 MB/s eta 0:00:01
     -------------------------------- ------ 42.7/50.7 MB 20.9 MB/s eta 0:00:01
     ---------------------------------- ---- 44.3/50.7 MB 19.7 MB/s eta 0:00:01
     ----------------------------------- --- 46.


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain_community.llms import LlamaCpp

llm = LlamaCpp(
    model_path=r"C:\Users\ASUS\Downloads\Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=-1,   # ✅ all layers on GPU
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=True       # set True to see loading logs
)

response = llm.invoke("Explain CUDA in one sentence.")
print(response)


llama_model_loader: loaded meta data with 23 key-value pairs and 195 tensors from C:\Users\ASUS\Downloads\Phi-3-mini-4k-instruct-fp16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                  phi3.attention.head_count u32              = 32
llama_model_loader: - kv   7:               phi3.attention.he

In [12]:
llm.invoke("Hi! My name is Maarten. What is 1 + 1?")

Llama.generate: 1 prefix-match hit, remaining 16 prompt tokens to eval
llama_perf_context_print:        load time =    1735.19 ms
llama_perf_context_print: prompt eval time =     648.52 ms /    16 tokens (   40.53 ms per token,    24.67 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     649.62 ms /    17 tokens
llama_perf_context_print:    graphs reused =          1


''

In [12]:
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.memory import ConversationBufferMemory

# Components
persona = "You are an expert in academic subjects and know everything in every field.\n"
instruction = "Use the conversation history if needed, answer the question asked and avoid repitative answer.\n"
context = "Your answer must be detailed and provide examples so the student can understand better.\n"
data_format = (
    "Structure your answer as follows:\n"
    "1. Adapt the detail of your answer to the complexity of the question.\n"
    "2. Start with a definition or direct answer in 1-2 sentences.\n"
    "3. Add 1-3 sentences of explanation with an example if relevant.\n"
    "4. End with a short bullet-point summary of key takeaways.\n"
    "Keep total length around 3–5 sentences plus the bullets.\n"
)
audience = "The answer is intended for students.\n"
tone = "The tone should be professional and clear.\n"
question_component = "Question to answer: {question}\n"

# Combine into a single template with placeholders
template = persona + instruction + context + data_format + audience + tone + question_component + \
           "Conversation history:\n{chat_history}\n<|assistant|>"

# Define PromptTemplate
prompt = PromptTemplate(
    template=template,
    input_variables=["question", "chat_history"]
)

# Conversation memory
memory = ConversationBufferMemory(memory_key="chat_history")

# Create the LLMChain
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)


In [21]:
llm_chain.invoke({
    "question":"Hi my name is ezam, provide me what is EDA in data analysis"
})

Llama.generate: 1 prefix-match hit, remaining 101 prompt tokens to eval
llama_perf_context_print:        load time =    1735.19 ms
llama_perf_context_print: prompt eval time =    3753.26 ms /   101 tokens (   37.16 ms per token,    26.91 tokens per second)
llama_perf_context_print:        eval time =   65829.27 ms /   327 runs   (  201.31 ms per token,     4.97 tokens per second)
llama_perf_context_print:       total time =   69843.64 ms /   428 tokens
llama_perf_context_print:    graphs reused =        324


{'question': 'Hi my name is ezam, provide me what is EDA in data analysis',
 'chat_history': '',
 'text': " Hello Ezam, EDA stands for Exploratory Data Analysis. It is a crucial step in the data analysis process where we visually and quantitatively inspect datasets to uncover patterns, relationships, or anomalies within the data. This approach helps us gain insights into the underlying structure of the data before proceeding with more advanced statistical methods or predictive modeling techniques.\n\nEDA involves a range of graphical tools such as histograms, scatter plots, box plots, and heat maps to explore different aspects of data like distributions, correlations between variables, trends, and outliers. For example, in analyzing customer satisfaction survey data with multiple questions, we can create a pair plot (a combination of scatter plots) to visualize how responses to each question are related or if there's any correlation between them.\n\nIn addition to these graphical metho

In [4]:
llm_chain.invoke({
    "question": "Can you explain me about jsp and servlet"
})

Llama.generate: 1 prefix-match hit, remaining 94 prompt tokens to eval
llama_perf_context_print:        load time =    1471.45 ms
llama_perf_context_print: prompt eval time =    3425.93 ms /    94 tokens (   36.45 ms per token,    27.44 tokens per second)
llama_perf_context_print:        eval time =  109342.83 ms /   499 runs   (  219.12 ms per token,     4.56 tokens per second)
llama_perf_context_print:       total time =  113307.51 ms /   593 tokens
llama_perf_context_print:    graphs reused =        490


{'question': 'Can you explain me about jsp and servlet',
 'chat_history': '',
 'text': " Certainly! JSP (JavaServer Pages) and Servlet are both server-side technologies used to develop dynamic web applications in Java. They work together within the Jakarta EE (formerly Java EE) ecosystem, allowing developers to create interactive and responsive user interfaces for web pages.\n\nJSP is a technology that helps developers generate HTML, XML or other text-based markup languages dynamically by embedding Java code directly into an HTML page. It uses special tags like <% %>, which are called JSP scripting directives, to insert dynamic content in the generated output. In essence, JSP combines static resources (HTML/CSS) with dynamic elements produced by executing Java code within those pages.\n\nFor example, consider a simple JSP file that displays user information:\n\n```jsp\n<!DOCTYPE html>\n<html>\n  <head>\n    <title>User Information</title>\n  </head>\n  <body>\n    <h1>Welcome, ${user.f

In [7]:
llm_chain.invoke({
    "question":"In 2025, who is Malaysia Prime Minister"
})

Llama.generate: 77 prefix-match hit, remaining 538 prompt tokens to eval
llama_perf_context_print:        load time =    1471.45 ms
llama_perf_context_print: prompt eval time =   21155.07 ms /   538 tokens (   39.32 ms per token,    25.43 tokens per second)
llama_perf_context_print:        eval time =   78147.89 ms /   344 runs   (  227.17 ms per token,     4.40 tokens per second)
llama_perf_context_print:       total time =   99598.84 ms /   882 tokens
llama_perf_context_print:    graphs reused =        382


{'question': 'In 2025, who is Malaysia Prime Minister',
 'chat_history': "Human: Can you explain me about jsp and servlet\nAI:  Certainly! JSP (JavaServer Pages) and Servlet are both server-side technologies used to develop dynamic web applications in Java. They work together within the Jakarta EE (formerly Java EE) ecosystem, allowing developers to create interactive and responsive user interfaces for web pages.\n\nJSP is a technology that helps developers generate HTML, XML or other text-based markup languages dynamically by embedding Java code directly into an HTML page. It uses special tags like <% %>, which are called JSP scripting directives, to insert dynamic content in the generated output. In essence, JSP combines static resources (HTML/CSS) with dynamic elements produced by executing Java code within those pages.\n\nFor example, consider a simple JSP file that displays user information:\n\n```jsp\n<!DOCTYPE html>\n<html>\n  <head>\n    <title>User Information</title>\n  </hea

In [13]:
llm_chain.invoke({
    "question":"what is python"
})

Llama.generate: 50 prefix-match hit, remaining 125 prompt tokens to eval
llama_perf_context_print:        load time =    1471.45 ms
llama_perf_context_print: prompt eval time =    4565.74 ms /   125 tokens (   36.53 ms per token,    27.38 tokens per second)
llama_perf_context_print:        eval time =   67455.41 ms /   314 runs   (  214.83 ms per token,     4.65 tokens per second)
llama_perf_context_print:       total time =   72342.50 ms /   439 tokens
llama_perf_context_print:    graphs reused =        313


{'question': 'what is python',
 'chat_history': '',
 'text': ' Python is a high-level, interpreted programming language known for its simplicity and readability. It was created by Guido van Rossum in the late 1980s and has since become one of the most popular languages worldwide.\n\nPython emphasizes code readability with its use of significant whitespace and avoidance of braces or semicolons, making it an excellent choice for beginners. It supports multiple programming paradigms including procedural, object-oriented, and functional programming approaches. An example of Python\'s simplicity can be seen in the following print statement:\n\n```python\nprint("Hello, World!")\n```\n\nPython has a large standard library that provides modules for various tasks such as web development (using frameworks like Django and Flask), data analysis (with libraries like Pandas and NumPy), and machine learning (through TensorFlow and Scikit-learn). Here is an example of using Python\'s requests library 

In [14]:
llm_chain.invoke({
    "question":"what is 1+100000"
})

Llama.generate: 165 prefix-match hit, remaining 342 prompt tokens to eval
llama_perf_context_print:        load time =    1471.45 ms
llama_perf_context_print: prompt eval time =   12557.16 ms /   342 tokens (   36.72 ms per token,    27.24 tokens per second)
llama_perf_context_print:        eval time =   43284.07 ms /   194 runs   (  223.11 ms per token,     4.48 tokens per second)
llama_perf_context_print:       total time =   55990.10 ms /   536 tokens
llama_perf_context_print:    graphs reused =        218


{'question': 'what is 1+100000',
 'chat_history': 'Human: what is python\nAI:  Python is a high-level, interpreted programming language known for its simplicity and readability. It was created by Guido van Rossum in the late 1980s and has since become one of the most popular languages worldwide.\n\nPython emphasizes code readability with its use of significant whitespace and avoidance of braces or semicolons, making it an excellent choice for beginners. It supports multiple programming paradigms including procedural, object-oriented, and functional programming approaches. An example of Python\'s simplicity can be seen in the following print statement:\n\n```python\nprint("Hello, World!")\n```\n\nPython has a large standard library that provides modules for various tasks such as web development (using frameworks like Django and Flask), data analysis (with libraries like Pandas and NumPy), and machine learning (through TensorFlow and Scikit-learn). Here is an example of using Python\'s r